<a href="https://colab.research.google.com/github/ZackWongAPO/Colab/blob/main/Taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#认识开发环境


##了解Colab
>Colab是一个在线笔记本，相当于OneNote的在线版本，并且可以将内容保存在GitHub账户上，主要支持Python系编程语言与多种文本注释语言，例如Markdown，LATEX，终端命令行等等，与线下Python编辑器不同，Colab允许用户对代码块进行分块执行，并且可以快速协作，不用不知道，一用是真香。
>>此页面并非Colab主页，只能在[Colab](https://colab.google/)中创建自己的笔记本，对大多数开源笔记本也只能保存副本至GitHub账户


###Colab上的磁盘空间
>>Colab上的文件运行是在云端虚拟机上，因此下载的各种Python库都下载到云端虚拟机上，与电脑关系不大，如果使用GPU加速，Colab将会分配更多的虚拟机内存，但是这还远远不够，如果要增加虚拟机内存只能购买ColabPro或者多开几个账号，与Google云盘的存储空间没有太大关系。通过以下代码可以实现将谷歌云中的文件保存在Colab虚拟机上[baidu](https://www.baidu.com)

In [ ]:
# 在content/drive文件夹下，两者文件共享
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###在Colab上使用GPU加速
>选择顶部工具栏中的`代码执行程序`子目录下的`更改运行时类型`中的`GPU`，从而使用云端图形加速。通过以下代码查看现在是否正在使用GPU加速

In [ ]:
!nvidia-smi

Wed Sep 25 02:10:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

##应用CUDA
>CUDA是NVIDIA公司开发的，能用GPU运算除了图形化处理之外的计算机算法的库。没有CUDA，GPU就只能用来打游戏，现在可以应用其来跑机器学习模型。以下代码就可以检测是否已经加载CUDA，如果此时使用CPU加速，则返回值应该是`False`,GPU加速的话应该返回的是`True`

In [ ]:
import torch as th
th.cuda.is_available()


True

##各Python机器学习库之间的关系
  >Keras, Tensorflow都是封装好的Python机器学习库
  >>Tensorflow相对底层，可以直接操作网络图和会话，上手难度较高，相当于机器学习领域的C语言。
  

>>Keras相对顶层，将各种复杂的参数传递过程封装起来，上手简单，相当于机器学习领域的Python。

#开始应用线性回归进行预测

##第一部分：加载所需模块和数据库

###第一步，在colab中加载所需要的Python库

In [ ]:
# 库的加载

# general
import io

# data processing
import numpy
import pandas

# machine learning support
import keras

# data visualization
import plotly.express as px
import plotly.subplots as sb
import plotly.graph_objects as go
import seaborn

###第二步加载数据库
>加载csv文件(*Comma-Separated Values, 用逗号分隔的纯文本文件，优点是占体积小，作为数据库来说易于下载*)至pandas数据格式(*把它变成让Python可以识别的表格的形式*)

In [ ]:
# load the dataset
chicago_taxi_dataset = pandas.read_csv("https://download.mlcc.google.com/mledu-datasets/chicago_taxi_train.csv")

###第三步从数据库中筛选所需要的属性值作为训练集


In [ ]:
# Updates dataframe to use specific columns
training_df = chicago_taxi_dataset[['TRIP_MILES', 'TRIP_SECONDS', 'FARE', 'COMPANY', 'PAYMENT_TYPE', 'TIP_RATE']]

print('Read dataset completed successfully.')
print('Total number of rows: {0}\n\n'.format(len(training_df.index)))
training_df.head(200)

Read dataset completed successfully.
Total number of rows: 31694




,TRIP_MILES,TRIP_SECONDS,FARE,COMPANY,PAYMENT_TYPE,TIP_RATE
0,2.57,2341,31.99,Flash Cab,Mobile,6.3
1,1.18,1074,9.75,Flash Cab,Credit Card,27.9
2,1.29,1173,10.25,Sun Taxi,Cash,0.0
3,3.70,3360,23.75,Choice Taxi Association,Cash,0.0
4,1.15,1044,10.00,Flash Cab,Cash,0.0
...,...,...,...,...,...,...
195,1.13,821,9.00,Blue Ribbon Taxi Association,Mobile,22.9
196,0.57,414,6.00,Flash Cab,Cash,0.0
197,1.22,886,9.00,City Service,Cash,0.0
198,1.68,1219,9.00,Sun Taxi,Mobile,23.0


##第二部分：数据挖掘